In [ ]:
from langgraph.graph import StateGraph

In [ ]:
class TreeState(MessagesState):
    # The unique ID
    id: uuid.UUID
    # The height of the current node
    height: int
    # The focused sub-problem
    focused_problem: str
    # The maximum initial height for expanding
    max_height: Optional[int]
    # Reason of associating this focused_problem
    # reason_of_associating: Optional[str]
    # Original problem that trigerred this problem
    original_problem: Optional[str]
    # Child list
    child_list: Optional[List["TreeState"]]
    # Responsd to this node as leaf
    respond: Optional[str]
    # Visualization info about this node
    artifact: Optional[Annotated[TableArtifact, ChartArtifact]]
    # Type of Visualization
    artifact_type: Optional[str]
    # Node to be expanded
    expanded_id: Optional[uuid.UUID]
    # Questions asked previously
    prev_questions: Optional[List[str]]
    # Regenerate time record, 0 means no need to regenerate
    regenerate_times: Optional[int]
    # num of subproblems need to generate
    max_subproblems: Optional[int]
    # tables in the dataset
    data_tables: Optional[str]
    # schema of data tables
    data_schemas: Optional[dict]
    # first level subproblems number
    first_level_subproblems: Optional[int]


In [ ]:
builder = StateGraph(TreeState)
builder.add_node("tree_traverser", traverse_tree)
builder.add_node("span", divide_problem)
builder.add_node("execute", solve_problem)
# builder.add_node("regenerate", regenerate_problem)
# builder.add_node("summarize", summarize_subproblems)
builder.add_node("tools_init", use_tools_init)
builder.add_node("tools_span", use_tools_span)
# builder.add_node("tools_regenerate", use_tools_regenerate)
builder.add_node("tools_execute", use_tools_execute)

builder.add_edge(START, "tree_traverser")
builder.add_conditional_edges("tree_traverser", lambda state: state["next"])
builder.add_conditional_edges(
    "span",
    route_span,
    {"span": "span", "tools_span": "tools_span", 'tree_traverser': 'tree_traverser'},
)
# builder.add_conditional_edges(
#     "regenerate",
#     route_regenerate,
#     {"regenerate": "regenerate", "tools_regenerate": "tools_regenerate", 'execute': 'execute'},
# )
builder.add_conditional_edges(
    "execute",
    route_execute,
    {"execute": "execute", "tools_execute": "tools_execute",
      'tree_traverser': 'tree_traverser'},# 'regenerate': 'regenerate'},
)

builder.add_edge("tools_init", "tree_traverser")
builder.add_edge("tools_span", "span")
# builder.add_edge("tools_regenerate", "regenerate")
builder.add_edge("tools_execute", "execute")